In [1]:
import pandas as pd
import os

In [3]:
path = os.getcwd()

'/Users/sb/Desktop/project/preference_rec/data'

ML-1m 

In [ ]:
column_names = ['UserID', 'MovieID', 'Rating', 'Timestamp']

df = pd.read_csv(f'{path}/ml-1m/ratings.dat', sep='::', header=None, names =column_names, engine= 'python')
df.to_csv(f'{path}/ml-1m/ratings.csv')

In [ ]:
column_names = ['MovieID','Title','Genres']
df = pd.read_csv(f'{path}/ml-1m/movies.dat', sep='::', header=None, names =column_names, engine= 'python',encoding='ISO-8859-1')

df.to_csv(f'{path}/data/ml-1m/movies.csv')

Amazon Beauty

In [ ]:
column_names = ['item', 'user','rating','timestamp']

df = pd.read_csv(f'{path}/beauty/All_Beauty.csv', sep=',',header=None, names=column_names, engine= 'python')
df.to_csv(f'{path}/beauty/Amazon_Beauty_small.csv')

Amazon Books

In [ ]:
column_names = ['item', 'user','rating','timestamp']

df = pd.read_csv(f'{path}/books/books.csv', sep=',',header=None, names=column_names, engine= 'python')
df.to_csv(f'{path}/books/books.csv')